In [12]:
from keras import backend as K
from keras.models import load_model
from keras.preprocessing import image
from keras.optimizers import Adam
from imageio import imread
import numpy as np
from matplotlib import pyplot as plt
import cv2
import time

from models.keras_ssd300 import ssd_300
from keras_loss_function.keras_ssd_loss import SSDLoss
from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_layers.keras_layer_DecodeDetectionsFast import DecodeDetectionsFast
from keras_layers.keras_layer_L2Normalization import L2Normalization

from ssd_encoder_decoder.ssd_output_decoder import decode_detections, decode_detections_fast

from data_generator.object_detection_2d_data_generator import DataGenerator
from data_generator.object_detection_2d_photometric_ops import ConvertTo3Channels
from data_generator.object_detection_2d_geometric_ops import Resize
from data_generator.object_detection_2d_misc_utils import apply_inverse_transforms

%matplotlib inline

img_height = 300
img_width = 300

K.clear_session() 

model = ssd_300(image_size=(img_height, img_width, 3),
                n_classes=1,
                mode='inference',
                l2_regularization=0.0005,
                scales=[0.1, 0.2, 0.37, 0.54, 0.71, 0.88, 1.05], 
                aspect_ratios_per_layer=[[1.0, 2.0, 0.5],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5],
                                         [1.0, 2.0, 0.5]],
                two_boxes_for_ar1=True,
                steps=[8, 16, 32, 64, 100, 300],
                offsets=[0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
                clip_boxes=False,
                variances=[0.1, 0.1, 0.2, 0.2],
                normalize_coords=True,
                subtract_mean=[123, 117, 104],
                swap_channels=[2, 1, 0],
                confidence_thresh=0.5,
                iou_threshold=0.45,
                top_k=200,
                nms_max_output_size=400)

#학습된 weight의 경로를 지정
weights_path = 'ssd300_pascal_07+12_epoch-04_loss-3.0387_val_loss-3.5244_weight.h5'

model.load_weights(weights_path, by_name=True)

adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)

model.compile(optimizer=adam, loss=ssd_loss.compute_loss)
#영상의 경로를 지정하고 프레임 캡쳐
vidcap = cv2.VideoCapture('C:\\Users\\user\Desktop\\video.mp4')
success,imagefile = vidcap.read()
count = 0
while success:
    if(count%5==0):
#프레임 캡쳐를 저장할 경로
        cv2.imwrite("C:\\Users\\user\Desktop\\frames\\frame%d.jpg" % count, imagefile)    
    success,imagefile = vidcap.read()
    count += 1

orig_images = [] 
input_images = [] 

for i in range(0,320):
    if(i%5==0):
#프레임 캡쳐를 저장하는 경로
        img_path = 'C:\\Users\\user\Desktop\\frames\\frame%d.jpg'%i
        #print(img_path)
        orig_images.append(imread(img_path))
        img = image.load_img(img_path, target_size=(img_height, img_width))
        img = image.img_to_array(img)
        img = np.array(img)
        input_images.append(img)
        
input_images = np.array(input_images)

num_of_frames = 16
counting = 0

print("Predicted boxes:\n")
print('   class   conf xmin   ymin   xmax   ymax')
    
for i in range(0, int(len(input_images)/num_of_frames)):
    y_pred = model.predict(input_images[i*num_of_frames:i*num_of_frames + num_of_frames])
    confidence_threshold = 0.4

    y_pred_thresh = [y_pred[k][y_pred[k,:,1] > confidence_threshold] for k in range(y_pred.shape[0])]
    np.set_printoptions(precision=2, suppress=True, linewidth=90)

    for j in range(0, num_of_frames):
        print('frame :',counting)
        counting += 5
#        print(y_pred_thresh[j])
        for box in y_pred_thresh[j]:
            # Transform the predicted bounding boxes for the 300x300 image to the original image dimensions.
            xmin = box[2] * orig_images[0].shape[1] / img_width
            ymin = box[3] * orig_images[0].shape[0] / img_height
            xmax = box[4] * orig_images[0].shape[1] / img_width
            ymax = box[5] * orig_images[0].shape[0] / img_height
            print('xmin : ',xmin, '  ymin : ',ymin, '  xmax : ',xmax, '  ymax : ',ymax)

Predicted boxes:

   class   conf xmin   ymin   xmax   ymax
frame : 0
frame : 5
frame : 10
frame : 15
frame : 20
frame : 25
frame : 30
frame : 35
frame : 40
frame : 45
frame : 50
frame : 55
frame : 60
frame : 65
frame : 70
frame : 75
frame : 80
frame : 85
frame : 90
frame : 95
frame : 100
xmin :  -3.7950758934020996   ymin :  11.739411900838217   xmax :  841.7351481119791   ymax :  216.10314778645832
frame : 105
frame : 110
frame : 115
xmin :  -9.302658081054688   ymin :  10.951525688171387   xmax :  834.99740234375   ymax :  211.03209197998046
frame : 120
xmin :  -7.218521931966146   ymin :  14.727643133799235   xmax :  831.1311800130209   ymax :  208.32061381022135
frame : 125
frame : 130
xmin :  0.18059635162353516   ymin :  14.443070087432861   xmax :  834.390283203125   ymax :  197.7364427693685
frame : 135
frame : 140
frame : 145
xmin :  -0.3447403907775879   ymin :  18.943992462158203   xmax :  834.871630859375   ymax :  201.02261678059895
frame : 150
xmin :  3.0437536239624023 